In [310]:
import collections
import re

import numpy as np
import pandas as pd
import tensorflow as tf

import nltk
from nltk.corpus import reuters
from nltk.tokenize import word_tokenize
from sklearn.manifold import TSNE
from sklearn import svm
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
%matplotlib inline

In [311]:
fb_posts = pd.read_pickle('../data/data_clean_4cols.pickle')

In [312]:
candidate_data = pd.DataFrame()
candidate_data = (fb_posts
                  .drop(['id', 'message'], axis=1)
                  .drop_duplicates('from_name')
                  .set_index('from_name'))

In [313]:
# join all messages by the same candidate
candidate_data['message'] = fb_posts.groupby('from_name')['message'].apply(' '.join)

#sample
candidate_data = candidate_data.sample(frac=0.2)

In [314]:
candidate_data

,Partei_ABK,message
from_name,,
Hubertus Heil,SPD,Auf dem Neujahrsempfang der Stadt Peine. Hier ...
Josef Ilsanker,DIE LINKE,"8. März 2016, Zeit danke zu sagen. 80% der Bes..."
Mahmut Özdemir,SPD,Dass dies der erste Post im neuen Jahr sein wü...
Ralph Lenkert,DIE LINKE,"Einfach und klar: gleiche Arbeit, gleicher Loh..."
Sigmar Gabriel,SPD,"Wieder #Istanbul, wieder starben Menschen, die..."
Dr. Jens Katzek,SPD,Kaum sind wir nach einer durchfeierten Nacht w...
Manja Schüle,SPD,"Ein glückliches, gesundes und friedliches neue..."
Jörn Wunderlich,DIE LINKE,So wird das Besucher- und Informationszentrum ...
"Konstantinos Katevas, Bundestagskandidat",DIE LINKE,PRESSEMITTEILUNG:\nDIE LINKE. im Landkreis Göp...


In [315]:
fb_posts = candidate_data

In [316]:
PERCENTAGE_DOCS = 100 # random subsample of Reuters training docs
VOCAB_SIZE = 10000
REMOVE_TOP_K_TERMS = 100
MIN_TERM_FREQ = 5

TEXT_WINDOW_SIZE = 8
BATCH_SIZE = 10 * TEXT_WINDOW_SIZE
EMBEDDING_SIZE = 128
SHUFFLE_EVERY_X_EPOCH = 5
PV_TEST_SET_PERCENTAGE = 5
NUM_STEPS = 100001
LEARNING_RATE = 0.1
NUM_SAMPLED = 64
REPORT_EVERY_X_STEPS = 2000

END_TO_END_EVERY_X_STEPS = 30000
E2E_TEST_SET_PERCENTAGE = 30
TSNE_NUM_DOCS = 400

# Token integer ids for special tokens
UNK = 0
NULL = 1

In [317]:
%load_ext line_profiler
%load_ext memory_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [318]:
"""
Returns an eternal generator, periodically shuffling the order

l_ is a list of integers; an internal copy of it is maintained.
"""
def repeater_shuffler(l_):
    l = np.array(l_, dtype=np.int32)
    epoch = 0
    while epoch >= 0:
        if epoch % SHUFFLE_EVERY_X_EPOCH == 0:
            np.random.shuffle(l)
        for i in l:
            yield i
        epoch += 1

In [319]:
def accept_doc(index):
    return np.random.random() * 100 < PERCENTAGE_DOCS

In [320]:
def accept(word):
    # Accept if not only Unicode non-word characters are present
    return re.sub(r'\W', '', word) != ''

In [321]:
def normalize(word):
    return word.lower()

In [322]:
def build_dataset():
    fileid2words = {fileid:
            [normalize(word) for word in word_tokenize(
                    row['message']) if accept(word)] \
            for fileid, row in fb_posts.iterrows() if accept_doc(fileid)}
    #import pdb; pdb.set_trace()
    count = [['__UNK__', 0], ['__NULL__', 0]]
    count.extend([(word, count) for word, count in collections.Counter(
            [word for words in fileid2words.values() \
            for word in words]).most_common(
                    VOCAB_SIZE - 2 + REMOVE_TOP_K_TERMS)[
                            REMOVE_TOP_K_TERMS:
                    ] if count >= MIN_TERM_FREQ])
    assert not set(['__UNK__', '__NULL__']) & set(next(zip(
            *count[2:])))
    dictionary = {}
    for i, (word, _) in enumerate(count):
        dictionary[word] = i
    reverse_dictionary = dict(zip(dictionary.values(),
                                  dictionary.keys()))
    data = []
    doclens = []
    fileids = []
    for docid, (fileid, words) in enumerate(fileid2words.items()):
        for word in words:
            if word in dictionary:
                wordid = dictionary[word]
            else:
                wordid = UNK
                count[UNK][1] += 1
            data.append((docid, wordid))
        # Pad with NULL values if necessary
        doclen = len(words)
        doclens.append(doclen)
        fileids.append(fileid)
        if doclen < TEXT_WINDOW_SIZE:
            n_nulls = TEXT_WINDOW_SIZE - doclen
            data.extend([(docid, NULL)] * n_nulls)
            count[NULL][1] += n_nulls
    return data, count, doclens, fileids, dictionary, reverse_dictionary

In [323]:
data, count, doclens, fileids, dictionary, reverse_dictionary = \
        build_dataset()

In [324]:
print('Number of documents:', len(set(next(zip(*data)))))
print('Number of tokens:', len(data))
print('Number of unique tokens:', len(count))
assert len(data) == sum([i for _, i in count])
print('Most common words (+UNK and NULL):', count[:5])
print('Least common words:', count[-5:])
print('Sample data:', data[:5])
vocab_size = min(VOCAB_SIZE, len(count))
print('Effective vocab size:', vocab_size)

Number of documents: 202
Number of tokens: 1558154
Number of unique tokens: 10000
Most common words (+UNK and NULL): [['__UNK__', 861845], ['__NULL__', 0], ('bundestag', 1694), ('vom', 1681), ('danke', 1664)]
Least common words: [('rwth', 11), ('entgeltgleichheitsgesetz', 11), ('ltwnrw', 11), ('musikalisch', 11), ('grenzregion', 11)]
Sample data: [(0, 0), (0, 0), (0, 643), (0, 0), (0, 88)]
Effective vocab size: 10000


In [325]:
pd.Series(doclens).describe()

count      202.000000
mean      7713.633663
std       6667.672624
min        153.000000
25%       2920.750000
50%       6663.500000
75%      10802.250000
max      56258.000000
dtype: float64

In [326]:
def get_text_window_center_positions():
    # If TEXT_WINDOW_SIZE is even, then define text_window_center
    # as left-of-middle-pair
    doc_start_indexes = [0]
    last_docid = data[0][0]
    for i, (d, _) in enumerate(data):
        if d != last_docid:
            doc_start_indexes.append(i)
            last_docid = d
    twcp = []
    for i in range(len(doc_start_indexes) - 1):
        twcp.extend(list(range(
                doc_start_indexes[i] + (TEXT_WINDOW_SIZE - 1) // 2,
                doc_start_indexes[i + 1] - TEXT_WINDOW_SIZE // 2
                )))
    return doc_start_indexes, twcp

In [327]:
doc_start_indexes, twcp = get_text_window_center_positions()

In [328]:
def get_train_test():
    split_point = len(twcp) * (100 - PV_TEST_SET_PERCENTAGE) // 100
    twcp_train = twcp[:split_point]

    # Test set data must come from known documents
    docids_train = set([data[i][0] for i in twcp_train])
    twcp_test = []
    for i in twcp[split_point:]:
        if data[i][0] in docids_train:
            twcp_test.append(i)
        else:
            twcp_train.append(i)
    if len(twcp_test) < (BATCH_SIZE // TEXT_WINDOW_SIZE):
        raise ValueError(
            'Too little test data, try increasing PV_TEST_SET_PERCENTAGE')
    return twcp_train, twcp_test

In [329]:
np.random.shuffle(twcp)
twcp_train, twcp_test = get_train_test()
twcp_train_gen = repeater_shuffler(twcp_train)
del twcp # save some memory

In [330]:
print('Effective test set percentage: {} out of {}, {:.1f}%'.format(
        len(twcp_test), len(twcp_test) + len(twcp_train),
        100 * len(twcp_test) / (len(twcp_test) + len(twcp_train))))

Effective test set percentage: 77603 out of 1552044, 5.0%


In [331]:
del twcp_train # save some memory, we use twcp_train_gen from now on

In [332]:
def generate_batch_single_twcp(twcp, i, batch, labels):
    tw_start = twcp - (TEXT_WINDOW_SIZE - 1) // 2
    tw_end = twcp + TEXT_WINDOW_SIZE // 2 + 1
    docids, wordids = zip(*data[tw_start:tw_end])
    batch_slice = slice(i * TEXT_WINDOW_SIZE,
                        (i+1) * TEXT_WINDOW_SIZE)
    batch[batch_slice] = docids
    labels[batch_slice, 0] = wordids
    
def generate_batch(twcp_gen):
    batch = np.ndarray(shape=(BATCH_SIZE,), dtype=np.int32)
    labels = np.ndarray(shape=(BATCH_SIZE, 1), dtype=np.int32)
    for i in range(BATCH_SIZE // TEXT_WINDOW_SIZE):
        generate_batch_single_twcp(next(twcp_gen), i, batch, labels)
    return batch, labels

In [333]:
# Input data
dataset = tf.placeholder(tf.int32, shape=[BATCH_SIZE])
labels = tf.placeholder(tf.int32, shape=[BATCH_SIZE, 1])

# Weights
embeddings = tf.Variable(
        tf.random_uniform([len(doclens), EMBEDDING_SIZE],
                          -1.0, 1.0))

softmax_weights = tf.Variable(
        tf.truncated_normal(
                [vocab_size, EMBEDDING_SIZE],
                stddev=1.0 / np.sqrt(EMBEDDING_SIZE)))
softmax_biases = tf.Variable(tf.zeros([vocab_size]))

# Model
# Look up embeddings for inputs
embed = tf.nn.embedding_lookup(embeddings, dataset)

In [334]:
# Compute the softmax loss, using a sample of the negative
# labels each time
loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(
                softmax_weights, softmax_biases, labels,
                embed, NUM_SAMPLED, vocab_size))

In [335]:
# Optimizer
optimizer = tf.train.AdagradOptimizer(LEARNING_RATE).minimize(
        loss)

# Test loss
test_loss = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = tf.matmul(embed, tf.transpose(
                          softmax_weights)) + softmax_biases,
                labels=labels[:, 0]))

# Normalized embeddings (to use cosine similarity later on)
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1,
                              keep_dims=True))
normalized_embeddings = embeddings / norm

In [336]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [337]:
def get_test_loss():
    # We do this in batches, too, to keep memory usage low.
    # Since our graph works with a fixed batch size, we
    # are lazy and just compute test loss on all batches that
    # fit in the test set.
    twcp_test_gen = (i for i in twcp_test)
    n_batches = (len(twcp_test) * TEXT_WINDOW_SIZE) // BATCH_SIZE
    test_l = 0.0
    for _ in range(n_batches):
        batch_data, batch_labels = generate_batch(twcp_test_gen)
        test_l += session.run([test_loss], feed_dict={
                dataset: batch_data, labels: batch_labels
            })[0]
    return test_l / n_batches

In [338]:
def train(num_steps):
    avg_training_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels = generate_batch(twcp_train_gen)
        _, l = session.run(
                [optimizer, loss],
                feed_dict={dataset: batch_data, labels: batch_labels})
        avg_training_loss += l
        if step > 0 and step % REPORT_EVERY_X_STEPS == 0:
            avg_training_loss = \
                    avg_training_loss / REPORT_EVERY_X_STEPS
            # The average loss is an estimate of the loss over the
            # last REPORT_EVERY_X_STEPS batches
            print('Average loss at step {:d}: {:.1f}'.format(
                    step, avg_training_loss))
            avg_training_loss = 0
            test_l = get_test_loss()
            print('Test loss at step {:d}: {:.1f}'.format(
                    step, test_l))

In [339]:
fb_posts

,Partei_ABK,message
from_name,,
Hubertus Heil,SPD,Auf dem Neujahrsempfang der Stadt Peine. Hier ...
Josef Ilsanker,DIE LINKE,"8. März 2016, Zeit danke zu sagen. 80% der Bes..."
Mahmut Özdemir,SPD,Dass dies der erste Post im neuen Jahr sein wü...
Ralph Lenkert,DIE LINKE,"Einfach und klar: gleiche Arbeit, gleicher Loh..."
Sigmar Gabriel,SPD,"Wieder #Istanbul, wieder starben Menschen, die..."
Dr. Jens Katzek,SPD,Kaum sind wir nach einer durchfeierten Nacht w...
Manja Schüle,SPD,"Ein glückliches, gesundes und friedliches neue..."
Jörn Wunderlich,DIE LINKE,So wird das Besucher- und Informationszentrum ...
"Konstantinos Katevas, Bundestagskandidat",DIE LINKE,PRESSEMITTEILUNG:\nDIE LINKE. im Landkreis Göp...


In [340]:
party_to_number = {'SPD': 1,
                   'CDU': 2,
                   'AfD': 3,
                   'DIE LINKE': 4,
                   'GRÜNE': 5,
                   'CSU': 6,
                   'FDP': 7}
e2e_labels = fb_posts['Partei_ABK'].map(party_to_number).as_matrix()
#fb_posts.as_matrix(columns=['Partei_ABK'])

In [341]:
target_names = 'SPD', 'CDU', 'AfD', 'DIE LINKE', 'GRÜNE', 'CSU', 'FDP'

In [342]:
e2e_labels

array([1, 4, 1, 4, 1, 1, 1, 4, 4, 2, 2, 1, 5, 2, 3, 7, 1, 2, 7, 6, 4, 1, 5,
       1, 5, 3, 7, 2, 4, 3, 7, 5, 1, 6, 1, 2, 4, 1, 2, 7, 2, 2, 7, 2, 1, 1,
       6, 2, 7, 1, 4, 1, 4, 2, 1, 7, 1, 1, 7, 2, 1, 6, 4, 3, 1, 2, 3, 4, 3,
       5, 7, 5, 1, 2, 1, 7, 3, 1, 4, 4, 1, 7, 4, 1, 5, 1, 2, 1, 3, 7, 7, 7,
       4, 3, 2, 4, 1, 5, 5, 3, 3, 1, 4, 5, 4, 4, 7, 1, 1, 7, 3, 3, 7, 2, 1,
       6, 2, 3, 5, 2, 6, 6, 3, 2, 6, 2, 4, 2, 7, 4, 3, 1, 4, 1, 2, 1, 1, 2,
       2, 1, 2, 4, 4, 5, 6, 4, 1, 5, 1, 5, 4, 4, 2, 1, 3, 1, 2, 7, 2, 2, 4,
       1, 2, 5, 6, 1, 7, 1, 3, 1, 1, 2, 6, 3, 7, 7, 5, 1, 4, 4, 2, 3, 4, 5,
       2, 7, 2, 7, 5, 3, 2, 1, 7, 7, 7, 7, 2, 1, 3, 5, 5, 1])

In [343]:
target_names

('SPD', 'CDU', 'AfD', 'DIE LINKE', 'GRÜNE', 'CSU', 'FDP')

In [344]:
def get_two_d_embeddings(embeddings):
    num_points = min(TSNE_NUM_DOCS, len(fileids))
    tsne = TSNE(perplexity=30, n_components=2, init='pca', 
                n_iter=5000)
    return tsne.fit_transform(embeddings[0:num_points, :])

In [345]:
def plot(embeddings):
    fig = plt.figure(figsize=(13, 8))
    class_1 = e2e_labels[:embeddings.shape[0]].astype('int')
    plt.plot(embeddings[class_1==1, 0], embeddings[class_1==1, 1], 
             'o', color='red')
    plt.plot(embeddings[class_1==2, 0], embeddings[class_1==2, 1],
             'o', color='black')
    plt.plot(embeddings[class_1==3, 0], embeddings[class_1==3, 1],
             'o', color='blue')
    plt.plot(embeddings[class_1==4, 0], embeddings[class_1==4, 1],
             'o', color='purple')
    plt.plot(embeddings[class_1==5, 0], embeddings[class_1==5, 1],
             'o', color='green')
    plt.plot(embeddings[class_1==6, 0], embeddings[class_1==6, 1],
             'o', color='lightblue')
    plt.plot(embeddings[class_1==7, 0], embeddings[class_1==7, 1],
             'o', color='black')
    plt.show()

In [346]:
def get_e2e_train_test():
    indices = np.array(range(len(fileids)), dtype=np.int32)
    np.random.shuffle(indices)
    split_point = len(indices) * (100 - E2E_TEST_SET_PERCENTAGE) \
            // 100
    e2e_train =np.array([True if i in indices[:split_point] \
                         else False for i in range(len(fileids))])
    return e2e_train

In [347]:
e2e_train = get_e2e_train_test()

In [348]:
def classification_experiment(embeddings):
    X = embeddings[e2e_train, :]
    y = e2e_labels[e2e_train]
    clf = svm.SVC(kernel='linear', class_weight='balanced')
    clf.fit(X, y)
    predictions = clf.predict(
            embeddings[~e2e_train, :])
    print(classification_report(
            e2e_labels[~e2e_train],
            predictions, target_names=target_names))

In [349]:
def end_to_end(num_steps):
    train(num_steps)
    current_embeddings = session.run(normalized_embeddings)
    plot(get_two_d_embeddings(current_embeddings))
    classification_experiment(current_embeddings)

In [ ]:
def run():
    for i in range(NUM_STEPS // END_TO_END_EVERY_X_STEPS):
        end_to_end(END_TO_END_EVERY_X_STEPS)
    end_to_end(NUM_STEPS % END_TO_END_EVERY_X_STEPS)

In [ ]:
#%mprun -f train run() # only works on physical file functions ;-(
%lprun -f train run() # uncomment to use (time profiling)
#%timeit -n 1 run() # uncomment to use (summary time profiling)
#%memit -r 1 run() # uncomment to use (memory consumption)

Average loss at step 2000: 3.1
